In [2]:
def plot(a1, a2, b1, b2):
    ax, fig = plt.subplots()

    ax = plt.plot(a1, a2, '-')
    ax = plt.plot(b1, b2, '--')
    ax = plt.scatter(a1, a2)
    ax = plt.scatter(b1, b2)

    fig.plot()

In [3]:
from scipy import integrate, interpolate
import math

columns = ('bitrate', 'Y_PSNR', 'U_PSNR', 'V_PSNR', 'YUV_PSNR')

def bdbr(HEVC,VVC,  yuv_sel = 1):
	HEVC = np.asarray(HEVC)
	VVC = np.asarray(VVC)
	
	HEVC = HEVC[HEVC[:,0].argsort()]
	VVC = VVC[VVC[:,0].argsort()]

	plot(pd.Series(np.log10(HEVC[:,0]), name='log_bitrate'), pd.Series(HEVC[:,yuv_sel], name='YUV_PSNR'), pd.Series(np.log10(VVC[:,0]), name='log_bitrate'), pd.Series(VVC[:,yuv_sel], name='YUV_PSNR'))
	plot(pd.Series(HEVC[:,0], name='log_bitrate')/1024, pd.Series(HEVC[:,yuv_sel], name='YUV_PSNR'), pd.Series(VVC[:,0], name='log_bitrate')/1024, pd.Series(VVC[:,yuv_sel], name='YUV_PSNR'))
	

	xa, ya = np.log10(HEVC[:,0]), HEVC[:,yuv_sel]
	xb, yb = np.log10(VVC[:,0]), VVC[:,yuv_sel]
	
	max_i = len(ya)
	i = 1
	while(i < max_i):
		if ya[i] < ya[i-1] or yb[i] <  yb[i-1]:
			ya = np.delete( ya,i)
			yb = np.delete( yb,i)
			xa = np.delete( xa,i)
			xb = np.delete( xb,i)
			max_i = len(ya)
		else:
			i += 1

	x_interp = [max(min(xa), min(xb)), min(max(xa),max(xb))]
	y_interp = [max(min(ya), min(yb)), min(max(ya),max(yb))]

	interp_br_a = interpolate.PchipInterpolator(ya,xa)
	interp_br_b = interpolate.PchipInterpolator(yb,xb)

	bdbr_a = integrate.quad(interp_br_a, y_interp[0], y_interp[1])[0]
	bdbr_b = integrate.quad(interp_br_b, y_interp[0], y_interp[1])[0]

	bdbr = (bdbr_b - bdbr_a) / (y_interp[1] - y_interp[0])
	bdbr = (math.pow(10., bdbr)-1)*100

	return bdbr

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import vvc_log as vl
import os

satds = [dir for dir in os.listdir('../outcsv') if os.path.isdir(os.path.join('../outcsv', dir))]
videos = [video for video in os.listdir('../out_preciso/Preciso/') if os.path.isdir(os.path.join('../out_preciso/Preciso', video))]
cfgs1 = ['intra', 'lowdelay', 'randomaccess']
cfgs2 = ['intra', 'low_delay', 'random_access']

videos.remove('RaceHorsesC')
videos.remove('FoodMarket')


In [8]:

files = []
for satd in satds:
    for video in videos:
        for i in range(len(cfgs1)):
            file1 = f'../out_preciso/Preciso/{video}/{cfgs2[i]}/Preciso{cfgs2[i]}{video}.csv'
            file2 = f'../outcsv/{satd}/csv/{cfgs1[i]}/{video}.csv'
            if not os.path.isfile(file1) or not os.path.isfile(file2):
                continue
            else:
                files.append((file2, file1))

df = {
    'satd': [],
    'video': [],
    'cfg': [],
    'bd_rate': []
}

for file in files:
    try:
        temp = vl.LogDF(file[0], file[1])
        df['satd'].append(temp.df['satd'][0])
        df['video'].append(temp.df['fileName'][0])
        df['cfg'].append(temp.df['encoder'][0])
        df['bd_rate'].append(temp.df['bd_rate'][0])
    except:
        continue


df = pd.DataFrame(df)    
print(df)
df.to_csv('bd_rate_data.csv', index = False)
df.to_excel('bd_rate_data.xlsx', index = False)
    
    
    







/home/devluis/Documentos/GitHub/VVC_research/source/vvc_log.py:173: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df['bd_rate'][ind] = bdbr


                satd            video           cfg   bd_rate
0    4x4-1-8x8-2.cpp       RaceHorses         intra  0.187221
1    4x4-1-8x8-2.cpp       RaceHorses      lowdelay  1.111627
2    4x4-1-8x8-2.cpp       RaceHorses  randomaccess  0.557779
3    4x4-1-8x8-2.cpp           BQMall         intra  0.085759
4    4x4-1-8x8-2.cpp           BQMall      lowdelay  0.582619
..               ...              ...           ...       ...
146            8x8-2   BasketballPass      lowdelay  0.327009
147            8x8-2   BasketballPass  randomaccess -0.213552
148            8x8-2  BasketballDrive         intra  0.112892
149            8x8-2  BasketballDrive      lowdelay  0.509534
150            8x8-2  BasketballDrive  randomaccess  0.094322

[151 rows x 4 columns]
